In [1]:
library(dplyr)
library(Seurat)
library(patchwork)


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


The legacy packages maptools, rgdal, and rgeos, underpinning this package
will retire shortly. Please refer to R-spatial evolution reports on
https://r-spatial.org/r/2023/05/15/evolution4.html for details.
This package is now running under evolution status 0 

Attaching SeuratObject



In [2]:
orthologs = read.table('../../data/ref/four_species_orthologous_protein_coding_genes_1to1_filtered.txt', sep='\t', header=1)

In [3]:
m1 <- readRDS('../../data/preprocessed/cell_level/human_M1_RNA_ATAC_ortho_ccres_unfiltered_5_3_22.rds')

In [4]:
DefaultAssay(m1) <- 'RNA'
m1[['ATAC']] <- NULL

Loading required package: Signac


Attaching package: ‘Signac’


The following object is masked from ‘package:Seurat’:

    FoldChange




In [5]:
anno <- read.table('../../data/preprocessed/cell_level/four_species_cell_labels_11_9_22.txt',header=T,row.names=1)
anno$species <- NULL

In [6]:
m1 <- AddMetaData(m1, anno, col.name='major_type')

In [7]:
unique(m1$orig.ident)

[1] "M1d1"     "M1d2"     "M1d3"     "M1d3MM"   "M1d1neun" "M1d2neun" "M1d3neun"

In [8]:
m1

An object of class Seurat 
36601 features across 60078 samples within 1 assay 
Active assay: RNA (36601 features, 0 variable features)

In [9]:
check = subset(m1, idents=c('M1d2neun'))

In [10]:
check

An object of class Seurat 
36601 features across 11659 samples within 1 assay 
Active assay: RNA (36601 features, 0 variable features)

In [11]:
m1 <- subset(m1, major_type != "N/A")

In [12]:
m1

An object of class Seurat 
36601 features across 40937 samples within 1 assay 
Active assay: RNA (36601 features, 0 variable features)

In [13]:
specie = 'human'
donor_out_f = "../../data/processed/gex_percent_expressed/%s/%s_donor_count_each_clust.tsv"

donors = unique(m1$orig.ident)
for (donor in donors){
    f = sprintf(donor_out_f, specie, donor)
    write.table(table(subset(m1, idents=c(donor))$major_type), file=f, sep='\t')
}

In [14]:
write.table(table(subset(m1, idents=c('M1d1'))$major_type), file="../../data/processed/gex_percent_expressed/all_donor_count_each_clust.tsv", sep='\t')

In [16]:
clusts = unlist(read.csv('../../data/ref/cluster_list.txt', header=FALSE)[[1]])

In [17]:
specie = 'human'

In [18]:
emptySparse <- function(nrow=0L, ncol=0L, format="R", dtype="d") {
    if (NROW(format) != 1L || !(format %in% c("R", "C", "T")))
        stop("'format' must be one of 'R', 'C', 'T'.")
    if (NROW(dtype) != 1L || !(dtype %in% c("d", "l", "n")))
        stop("'dtype' must be one of 'd', 'l', 'n'.")
    nrow <- as.integer(nrow)
    ncol <- as.integer(ncol)
    if (NROW(nrow) != 1L || is.na(nrow) || nrow < 0)
        stop("'nrow' must be a non-negative integer.")
    if (NROW(ncol) != 1L || is.na(ncol) || ncol < 0)
        stop("'ncol' must be a non-negative integer.")
    target_class <- sprintf("%sg%sMatrix", dtype, format)
    out <- new(target_class)
    out@Dim <- as.integer(c(nrow, ncol))
    if (format == "R") {
        out@p <- integer(nrow+1L)
    } else if (format == "C") {
        out@p <- integer(ncol+1L)
    }
    return(out)
}

In [19]:
above_threshold_counts <- function(genes, data) {
    # geg dataframe where the gene is
    # for each i
    datas = list()
    donors = unique(data$orig.ident)
    for (donor in donors){
        use = subset(x=data, idents = donor)
        tmp = n_cell_expressed(use, genes, group="major_type")
        datas = append(datas, list(tmp))
    }
    names(datas) <- donors
    return(datas)
}
n_cell_expressed <- function (object, genes, group){
    datas = SplitObject(object=object, split.by=group)
    vals = lapply(datas, n_expressed, genes)
    vals = do.call(cbind, vals)
    return(vals)
}
n_expressed <- function(object, genes){
    counts = object[['RNA']]@counts[genes, ]
    empty = emptySparse(nrow = length(genes), ncol=2)
    temp = do.call(cbind, c(counts, empty))
    # if(genes %in% row.names(counts)){
    return(Matrix::rowSums(temp > 0))
    # }else{return(NA)}
}

In [20]:
test = rownames(m1)[c(50, 23, 97, 188, 199, 2034, 3024, 5032)]

In [21]:
x = above_threshold_counts(test, m1)

In [22]:
write_tables <- function(data, out_f){
    for (name in names(data)){
        dat = data[[name]]
        f = sprintf(out_f, name)
        write.table(dat, file=f, sep='\t')
    }
}

In [23]:
donor_out_f = "../../data/processed/gex_percent_expressed/%s/%s_donor_n_expressed_each_clust.tsv"

# loop per species

In [ ]:
species = c('human', 'macaque', 'marmoset', 'mouse')

In [ ]:
data_store = list(human = 'human_M1_RNA_ATAC_ortho_ccres_unfiltered_5_3_22.rds', 
                  macaque = 'macaque_m1_rna_filtered_merged_GCF_10_4_22.rds',
                  marmoset = 'marmoset_M1_unfiltered_ortho_ccres_5_3_22.rds',
                  mouse = 'mop_multiome_ortho_ccres_5_3_22.rds')

In [ ]:
data_store[['human']]

In [ ]:
species_in_f = '../../data/preprocessed/cell_level/%s'

In [ ]:
start_time <- Sys.time()

for (specie in species[2:4]){
    # set defaults
    genes = orthologs[[sprintf('%s_gene', specie)]]
    
    in_f = sprintf(species_in_f, data_store[[specie]])
    out_f = sprintf(donor_out_f, specie, '%s')
    # load dataset
    m1 = readRDS(in_f)
    DefaultAssay(m1) <- 'RNA'
    if (specie == 'human'){
        m1[['ATAC']] <- NULL
    }
    check_genes = rownames(m1)
    use_genes = intersect(check_genes, genes)
    
    # add cell type levels
    m1 <- AddMetaData(m1, anno, col.name='major_type')
    # remove missing cell labels
    m1 <- subset(m1, major_type != "N/A")
    if(specie == 'macaque'){
        m1 <- SetIdent(m1, value = "orig.ident")
    }
    # save cell type abundance for each donor
    out_f_ = "../../data/processed/gex_percent_expressed/%s/%s_donor_count_each_clust.tsv"
    donors = unique(m1$orig.ident)
    for (donor in donors){
        f = sprintf(out_f_, specie, donor)
        write.table(table(subset(m1, idents=c(donor))$major_type), file=f, sep='\t')
    }
    cells_expressed = above_threshold_counts(use_genes, m1)
    write_tables(data = cells_expressed, out_f=out_f)
}
end_time <- Sys.time()
end_time - start_time

In [ ]:
specie